## Loading all data and labels from file

In [242]:
# Loading all data and labels from file
import h5py
import numpy as np
from keras.utils.np_utils import to_categorical

feature_file = "../preprocess/MSRII-c3d-features.h5"
mode = "r"
with h5py.File(feature_file, mode) as h5:
        '''
        print(h5.keys())
        print(h5['features'])
        print(h5['labels'])
        '''
        data = np.array(h5['features'])
        labels = np.array(h5['labels'])
        labels[np.where(labels > 0)] = 1
        labels = to_categorical(labels, 2)

print(data.shape)
print(labels.shape)

# Seperate data and labels into one training set and one test set
data_rows, _ = data.shape
print(data_rows)

np.random.seed(1993)
rand_permed_rows_seed = np.random.permutation(data_rows)
print(rand_permed_rows_seed)

rand_permed_data = data[rand_permed_rows_seed]
rand_permed_labels = labels[rand_permed_rows_seed]

test_data_rows = 500
X_test = rand_permed_data[0:test_data_rows, :]
y_test = rand_permed_labels[0:test_data_rows, :]
X_train = rand_permed_data[test_data_rows: , :]
y_train = rand_permed_labels[test_data_rows: , :]


(2359, 4096)
(2359, 2)
2359
[1160  640 2349 ..., 1969 1721 1441]


## MLP model

In [243]:
# MLP model (with tanh sigmoid layers)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Create model
model = Sequential()

# Dense(#) is a fully-connected layer with [#] hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 4096-dimensional vectors.
model.add(Dense(256, input_dim=4096, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(256, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(256, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(2, init='uniform'))
model.add(Activation('softmax'))


# Compile model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_95 (Dense)                 (None, 256)           1048832     dense_input_37[0][0]             
____________________________________________________________________________________________________
activation_95 (Activation)       (None, 256)           0           dense_95[0][0]                   
____________________________________________________________________________________________________
dropout_59 (Dropout)             (None, 256)           0           activation_95[0][0]              
____________________________________________________________________________________________________
dense_96 (Dense)                 (None, 256)           65792       dropout_59[0][0]                 
___________________________________________________________________________________________

In [244]:
batch_size = 16
nb_epoch = 1000

best_accuracy = 0.0
best_epoch = 0
np.random.seed(1993)
for ep in range(1, nb_epoch):
    H = model.fit(X_train,
                  y_train,
                  batch_size=batch_size,
                  nb_epoch=1, verbose=0,
                  validation_split=0.2)
    loss = H.history['val_loss']
    accuracy = H.history['val_acc']
    
    print('epoch\t' + str(ep) + ':\t(loss, accuracy) = ' + str(loss) + ',\t' + str(accuracy))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = ep
        best_W = model.get_weights()


print('-------------------------------------------------------------------------------')
print('best_accuracy: ' + str(best_accuracy) + ' generated at epoch ' + str(best_epoch))
        
model.reset_states()
model.set_weights(best_W)


epoch	1:	(loss, accuracy) = [0.42142141121689991],	[0.82795698924731187]
epoch	2:	(loss, accuracy) = [0.39969678351315119],	[0.83064516129032262]
epoch	3:	(loss, accuracy) = [0.32082391498229834],	[0.84408602150537637]
epoch	4:	(loss, accuracy) = [0.339049643406304],	[0.88709677419354838]
epoch	5:	(loss, accuracy) = [0.3716523100932439],	[0.86559139784946237]
epoch	6:	(loss, accuracy) = [0.28076701995826536],	[0.88709677419354838]
epoch	7:	(loss, accuracy) = [0.34247423884689165],	[0.83333333333333337]
epoch	8:	(loss, accuracy) = [0.36123437939151642],	[0.81989247311827962]
epoch	9:	(loss, accuracy) = [0.31102135072472276],	[0.87634408602150538]
epoch	10:	(loss, accuracy) = [0.32850412175219545],	[0.86827956989247312]
epoch	11:	(loss, accuracy) = [0.38395608929536673],	[0.80107526881720426]
epoch	12:	(loss, accuracy) = [0.28270673607626268],	[0.89247311827956988]
epoch	13:	(loss, accuracy) = [0.34698003434365793],	[0.84946236559139787]
epoch	14:	(loss, accuracy) = [0.29556079513283184]

In [245]:
# Test
loss, accuracy = model.evaluate(X_test, y_test, batch_size=16)
accuracy


352/500 [====================>.........] - ETA: 0s

0.91200000000000003